In [1]:
img_path = 'data/sampel.jpg'

In [2]:
from FeatureExtraction import deteksiPerson, kedalamanCitra, ekstrakMetadataCitra, ekstraksiFitur

/home/anugerahsurya/tinggikuenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import pipeline
# Load depth estimation pipeline sekali di luar fungsi untuk efisiensi
depth_estimator = pipeline(task="depth-estimation", model="Intel/zoedepth-nyu-kitti")
dataObjek, kedalamanObjek, metadata = ekstraksiFitur(img_path, depth_estimator)

NameError: name 'pipeline' is not defined